In [4]:
import ast
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from collections import Counter
import statsmodels
import datetime
from sklearn.model_selection import train_test_split

df = pd.read_csv("../metadata/ted_main.csv")
df['film_date'] = df['film_date'].apply(lambda x: datetime.datetime.fromtimestamp( int(x)))
df['published_date'] = df['published_date'].apply(lambda x: datetime.datetime.fromtimestamp( int(x)))
df['film_year'] = df['film_date'].apply(lambda it: it.year)
df['published_year'] = df['published_date'].apply(lambda it: it.year)
df

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,film_year,published_year
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,2006-02-25 02:00:00,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,2006-06-27 03:11:00,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,2006,2006
1,265,With the same humor and humanity he exuded in ...,977,TED2006,2006-02-25 02:00:00,43,Al Gore,Al Gore: Averting the climate crisis,1,2006-06-27 03:11:00,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,2006,2006
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,2006-02-24 02:00:00,26,David Pogue,David Pogue: Simplicity sells,1,2006-06-27 03:11:00,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,2006,2006
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,2006-02-26 02:00:00,35,Majora Carter,Majora Carter: Greening the ghetto,1,2006-06-27 03:11:00,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,2006,2006
4,593,You've never seen data presented like this. Wi...,1190,TED2006,2006-02-22 02:00:00,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,2006-06-27 23:38:00,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,2006,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,2017-06-06 03:00:00,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,2017-09-19 23:00:16,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430,2017,2017
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,2017-04-24 03:00:00,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,2017-09-20 18:02:17,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470,2017,2017
2547,10,Science fiction visions of the future show us ...,651,TED2017,2017-04-24 03:00:00,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,2017-09-21 18:01:35,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647,2017,2017
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,2017-07-08 03:00:00,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man g

In [5]:
# Filter dataset

print(f"Total talks between 2010-2016: {len(df.loc[(df['film_year'] >= 2010) & (df['film_year'] <= 2016), :])}")

# We will use videos between 2010-2016
df = df.loc[(df['film_year'] >= 2010) & (df['film_year'] <= 2016), :]
# Num_speakers == 1
df = df.loc[df['num_speaker'] == 1, :] 
print(len(df))

Total talks between 2010-2016: 1796
1758


# Views target set

In [31]:
import plotly.express as px
    
log_views = np.log(df['views'])
df['log_views_norm'] = (log_views - log_views.mean()) / log_views.std()
px.histogram(df['log_comments_norm']).show()

log_comments = np.log(df['comments'])
df['log_comments_norm'] = (log_comments - log_comments.mean()) / log_comments.std()
px.histogram(df['log_views_norm']).show()

comments_per_view = df['comments'] / df['views']
log_comments_per_view = np.log(comments_per_view)
df['log_comments_per_view_norm'] = (log_comments_per_view - log_comments_per_view.mean()) / log_comments_per_view.std()
px.histogram(df['log_comments_per_view_norm']).show()


In [32]:
from functools import partial

def map_to_label(value, ranges, labels):
    if len(ranges) != len(labels):
        raise ValueError("Number of labels must be equal to the number of ranges")

    for range, label in zip(ranges, labels):
        if value >= range[0] and value <= range[1]:
            return label

    raise ValueError(f"value {value} not located between ranges: {ranges}")

views_mapping_5 = partial(map_to_label,
                          ranges=[[-np.inf, -1.5], [-1.5, -0.5], [-0.5, 0.5], [0.5, 1.5], [1.5, np.inf]],
                          labels=["very_low", "low", "medium", "high", "very_high"])

views_mapping_3 = partial(map_to_label,
                          ranges=[[-np.inf, -0.5], [-0.5, 0.5], [0.5, np.inf]],
                          labels=["low", "medium", "high"])



df['log_views_norm_cat'] = df['log_views_norm'].apply(views_mapping_3)
df['log_comments_norm_cat'] = df['log_comments_norm'].apply(views_mapping_3)
df['log_comments_per_view_norm_cat'] = df['log_comments_per_view_norm'].apply(views_mapping_3)

px.histogram(df['log_views_norm_cat']).show()
px.histogram(df['log_comments_norm_cat']).show()
px.histogram(df['log_comments_per_view_norm_cat']).show()